# Questrade Portfolio Overview

In [1]:
import os
import math
import price
import requests
from questrade import qbot
import numpy as np
import pandas as pd
import datetime as dt
import matplotlib.pyplot as plt
import pandas_datareader.data as web
#from questrade_api import Questrade

In [2]:
questrade = qbot()

HTTPError: 401 Client Error: Unauthorized for url: https://api03.iq.questrade.com/v1/accounts

## Account Balance Summary

In [ ]:
balance = questrade.get_balance()
balance

## Position & Performance Summary

In [ ]:
portfolio = questrade.get_positions()

In [ ]:
portfolio['%Portfolio'].plot(kind='pie',
                             title='Portfolio Breakdown',
                             figsize=(10, 10),
                             legend=True)

In [ ]:
portfolio = portfolio.style.applymap(questrade.color_negative_red, subset=['Gain/Loss (%)'])
portfolio

In [ ]:
holdings = list(portfolio.index)
holdings

In [ ]:
data = {'Symbol': [], 'Dividend_Growth': [], 'Momentum': []}
MOMENTUM_PERIODS = [3,6,12,24,36,48,60]
for symbol in holdings:
    data['Symbol'].append(symbol)
    try:
        div_growth = dividend.calcualte_avg_dividend_growth(symbol,10)
        if div_growth == float('inf'):
            data['Dividend_Growth'].append(np.nan)
        else:
            data['Dividend_Growth'].append(div_growth)
    except:
        data['Dividend_Growth'].append(np.nan)
        
    try:
        mom = momentum.calculate_equal_weight_momentum(symbol, MOMENTUM_PERIODS)
        data['Momentum'].append(mom)
    except:
        data['Momentum'].append(np.nan)

print(data)

df = pd.DataFrame(data)
df.dropna(inplace=True)
df

## Monthly Dividend Income

In [ ]:
div = questrade.get_dividend_income()
div.loc['Total'] = div.sum(numeric_only=True, axis=0)
div

In [ ]:
div[:-1].plot(kind='bar',
                    title='Monthly Dividend Payments',
                    xlabel='Months',
                    ylabel='Dividend ($)',
                    figsize=(15, 10),
                    legend=False,
                    ylim=(0, 30))

## Global Macro Momentum

global_macro = ['SPY', 'QQQ', 'TLT', 'IEF', 'GLD', 'DBC']
period = [1,3,6,12,24,36,48,60]

data = {'Symbol': [], 'Momentum': []}
count = 0
for symbol in global_macro:
    count += 1
    print(f"{count}/{len(global_macro)}")
    data['Symbol'].append(symbol)
    data['Momentum'].append(momentum.calculate_equal_weight_momentum(symbol, period))
momentum_df = pd.DataFrame(data)
momentum_df.set_index('Symbol')
momentum_df.sort_values(by='Momentum',inplace=True, ascending=False)
momentum_df

## US Sector Momentum

sector_df = pyticker.get_sector_df()
period = [1,3,6,12]
count = 0
for x in list(sector_df.index):
    count += 1
    print(f"{count}/{len(list(sector_df.index))}")
    sector_df.loc[x,'Momentum'] = momentum.calculate_equal_weight_momentum(x, period)

sector_df.sort_values(by='Momentum', inplace=True, ascending=False)
sector_df